<a href="https://colab.research.google.com/github/jcmeve/MP/blob/main/MP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from IPython.core.display import Math
import numpy as np
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from torch import nn
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
import json

In [ ]:
from google.colab import drive
import pandas as pd
drive.mount('/content/drive')
filename = '/content/drive/MyDrive/Colab Notebooks/total_score.csv'
data = pd.read_csv(filename)
data = data.sample(frac=1).reset_index(drop=True) 
data.head()




Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,documents__category_group_code,documents__category_group_name,documents__category_name,documents__distance,time,score(0~10)
0,AT4,관광명소,"여행 > 관광,명소 > 계곡",1163,오후,8
1,CT1,문화시설,"문화,예술 > 문화시설 > 전시관",437,밤,8
2,CE7,카페,음식점 > 카페,2525,밤,1
3,FD6,음식점,"음식점 > 한식 > 육류,고기 > 갈비",4662,밤,1
4,CE7,카페,음식점 > 카페,1031,저녁,2


In [ ]:
from sklearn.model_selection import train_test_split

X = data.drop('score(0~10)', axis=1)
y = data['score(0~10)']
  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)
print("X_train's shape : ", X_train.shape)
print("y_train's shape : ", y_train.shape)
print("\nX_test's shape : ", X_test.shape)
print("y_test's shape : ", y_test.shape)




X_train's shape :  (1458, 5)
y_train's shape :  (1458,)

X_test's shape :  (162, 5)
y_test's shape :  (162,)


In [ ]:

class ContextDataSet(Dataset):
  def __init__(self, target_type, time, before_distance, score):
    ###one hot encoding

    type_converter = {'AT4':0,'MT1':1,'CT1':2,'AD5':3,'FD6':4, 'CE7':5}
    self.target_type = []
    for i in range(len(target_type)):
      self.target_type.insert(i,type_converter[target_type[i]])


    time_converter = {'아침':0,'오전':1,'점심':2,'오후':3,'저녁':4,'밤':5 }
    self.time = []
    for i in range(len(time)):
      self.time.insert(i, time_converter[time[i]])



    self.before_distance = before_distance
    self.score = score


    self.target_type = nn.functional.one_hot(torch.tensor(self.target_type),6)
    self.time = nn.functional.one_hot(torch.tensor(self.time),6)


      


  def __len__(self):
    return len(self.score)
  def __getitem__(self,idx):
    feature = []

    for i in range(len(self.target_type[idx])):
      feature.append(self.target_type[idx][i])
    for i in range(len(self.time[idx])):
      feature.append(self.time[idx][i])
    feature.append(self.before_distance[idx]*0.0001)
    

    return feature, self.score[idx]

In [ ]:

train_set = ContextDataSet(
    X_train['documents__category_group_code'].values.tolist(),
    X_train['time'].values.tolist(),
    X_train['documents__distance'].values.tolist(),
    y_train.to_list()
    )
train_loader = DataLoader(train_set, batch_size = 1, shuffle = True)

test_set = ContextDataSet(
    X_test['documents__category_group_code'].values.tolist(),
    X_test['time'].values.tolist(),
    X_test['documents__distance'].values.tolist(),
    y_test.to_list()
    )
test_loader = DataLoader(test_set, batch_size = 1, shuffle = True)

In [ ]:
feature,lable = next(iter(train_loader))
print(feature, lable)

[tensor([0]), tensor([0]), tensor([0]), tensor([0]), tensor([1]), tensor([0]), tensor([0]), tensor([0]), tensor([0]), tensor([0]), tensor([0]), tensor([1]), tensor([0.0040], dtype=torch.float64)] tensor([5])


In [ ]:
### 날씨는 ai안쓰고 거름

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Linear(in_features=13, out_features=4, bias = False),
            nn.Sigmoid(),
        )

        self.layer2 = nn.Sequential(
            nn.Linear(in_features=4, out_features=1, bias = False),
        )

    def forward(self, x):
        x = x.to(device)
        logits = self.layer1(x)
        logits = self.layer2(logits)
        return logits 

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

Using cpu device


In [ ]:
model = NeuralNetwork().to(device) # device로 Network 전송
print(model) # 모델 구조 확인

NeuralNetwork(
  (layer1): Sequential(
    (0): Linear(in_features=13, out_features=4, bias=False)
    (1): Sigmoid()
  )
  (layer2): Sequential(
    (0): Linear(in_features=4, out_features=1, bias=False)
  )
)


In [ ]:
# hyperparameter 설정
import torch.optim as optim

#criterion = nn.CrossEntropyLoss() # loss function
criterion = nn.MSELoss() # loss function
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9) # optimizer

n_epoch = 50  # the number of epochs
n_batch = 10 # the number of batches

In [ ]:
for epoch in range(n_epoch):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):

        inputs, labels = data
        
        inputs = torch.tensor([inputs], dtype = torch.float32)
        labels = torch.tensor([labels], dtype = torch.float32)

        inputs = inputs.to(device)
        labels = labels.to(device)

        # optimizer의 파라미터 gradient를 0으로 설정
        optimizer.zero_grad()

        pred = model(inputs).to(device)#예측값 forward

        loss = criterion(pred, labels) #loss값

        loss.backward() #backward
        optimizer.step() #optimize
        #############################################

        # loss 출력
        running_loss += loss.item()
        
        if i % n_batch == 0:    # print every n_batch mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / n_batch:.3f}')
            running_loss = 0.0

print('Finished Training')

[1,     1] loss: 0.321
[1,    11] loss: 9.495
[1,    21] loss: 17.908
[1,    31] loss: 14.276
[1,    41] loss: 11.792
[1,    51] loss: 9.532
[1,    61] loss: 2.750
[1,    71] loss: 11.298
[1,    81] loss: 3.820
[1,    91] loss: 12.620
[1,   101] loss: 9.291
[1,   111] loss: 14.004


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:530: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
[16,  1111] loss: 3.122
[16,  1121] loss: 0.725
[16,  1131] loss: 4.669
[16,  1141] loss: 3.699
[16,  1151] loss: 2.609
[16,  1161] loss: 0.836
[16,  1171] loss: 1.150
[16,  1181] loss: 2.018
[16,  1191] loss: 1.683
[16,  1201] loss: 2.452
[16,  1211] loss: 1.762
[16,  1221] loss: 1.915
[16,  1231] loss: 1.241
[16,  1241] loss: 0.511
[16,  1251] loss: 1.302
[16,  1261] loss: 1.811
[16,  1271] loss: 3.756
[16,  1281] loss: 1.303
[16,  1291] loss: 1.168
[16,  1301] loss: 0.459
[16,  1311] loss: 3.398
[16,  1321] loss: 2.112
[16,  1331] loss: 2.666
[16,  1341] loss: 3.752
[16,  1351] loss: 2.470
[16,  1361] loss: 0.532
[16,  1371] loss: 0.340
[16,  1381] loss: 1.356
[16,  1391] loss: 1.537
[16,  1401] loss: 0.932
[16,  1411] loss: 2.781
[16,  1421] loss: 2.255
[16,  1431] loss: 1.221
[16,  1441] loss: 0.823
[16,  1451] loss: 1.368
[17,     1] loss: 0.570
[17,    11] loss: 0.462
[17,    21] loss: 2.160
[17,    31] loss: 1.055
[17,    41] loss: 2.481
[17,

In [ ]:
# 전체 test data에 대한 결과 확인
correct = 0
total = 0

diff_sum = 0

with torch.no_grad(): # 모델을 학습하는 것이 아니므로 gradient 계산을 할 필요가 없음
    for data in test_loader:
        inputs, labels = data

        inputs = torch.tensor([inputs], dtype = torch.float32)
        labels = torch.tensor([labels], dtype = torch.float32)
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs)

        #_, predicted = torch.max(outputs.data, 1)

        predicted = outputs
        print(predicted.item())
        print(labels.item())
        diff = predicted.item() - labels.item()
        diff_sum+= diff

        total += labels.size(0)


        correct += (predicted == labels.to(device)).sum().item()

print(f'Accuracy of the network on the test images: {100 * correct // total} %')
diff_avg = diff_sum / total
print(diff_avg)

0.4612365663051605
1.0
1.2123075723648071
1.0
0.20822837948799133
0.0
0.07646489888429642
0.0
8.324188232421875
9.0
1.07477605342865
2.0
1.7149381637573242
2.0
1.268341302871704
3.0
4.862968444824219
2.0
7.362471103668213
6.0
2.2461259365081787
3.0
5.960259437561035
2.0
0.7898714542388916
1.0
1.4030325412750244
1.0
0.5166954398155212
1.0
0.15697550773620605
0.0
0.22996291518211365
0.0
3.9814565181732178
6.0
2.727924346923828
3.0
1.3543052673339844
0.0
5.7391815185546875
3.0
7.510005950927734
9.0
3.321561098098755
3.0
2.3127152919769287
4.0
1.652605414390564
2.0
3.3258237838745117
2.0
2.1909308433532715
2.0
1.4479624032974243
1.0
1.6063296794891357
1.0
8.475870132446289
8.0
6.155653476715088
7.0
0.21297156810760498
0.0
2.9249038696289062
1.0
0.0794609934091568
0.0
1.3663731813430786
1.0
2.0804154872894287
1.0
7.301174163818359
8.0
2.106532573699951
5.0
8.017374992370605
10.0
5.970448017120361
5.0
1.2296702861785889
0.0
2.265995979309082
2.0
5.267245292663574
5.0
0.20142176747322083
0.0


In [ ]:
!curl -v -X GET "https://dapi.kakao.com/v2/local/search/category.json?category_group_code=AT4&radius=10000&y=33.4889179032603&x=126.498229141199" -H "Authorization: KakaoAK 9a12f840d08a0b698f040ffc93f18211" > AT4
!curl -v -X GET "https://dapi.kakao.com/v2/local/search/category.json?category_group_code=MT1&radius=10000&y=33.4889179032603&x=126.498229141199" -H "Authorization: KakaoAK 9a12f840d08a0b698f040ffc93f18211" > MT1
!curl -v -X GET "https://dapi.kakao.com/v2/local/search/category.json?category_group_code=CT1&radius=10000&y=33.4889179032603&x=126.498229141199" -H "Authorization: KakaoAK 9a12f840d08a0b698f040ffc93f18211" > CT1
!curl -v -X GET "https://dapi.kakao.com/v2/local/search/category.json?category_group_code=AD5&radius=10000&y=33.4889179032603&x=126.498229141199" -H "Authorization: KakaoAK 9a12f840d08a0b698f040ffc93f18211" > AD5
!curl -v -X GET "https://dapi.kakao.com/v2/local/search/category.json?category_group_code=FD6&radius=10000&y=33.4889179032603&x=126.498229141199" -H "Authorization: KakaoAK 9a12f840d08a0b698f040ffc93f18211" > FD6
!curl -v -X GET "https://dapi.kakao.com/v2/local/search/category.json?category_group_code=CE7&radius=10000&y=33.4889179032603&x=126.498229141199" -H "Authorization: KakaoAK 9a12f840d08a0b698f040ffc93f18211" > CE7

Note: Unnecessary use of -X or --request, GET is already inferred.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0*   Trying 211.249.220.147...
* TCP_NODELAY set
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0* Connected to dapi.kakao.com (211.249.220.147) port 443 (#0)
* ALPN, offering h2
* ALPN, offering http/1.1
* successfully set certificate verify locations:
*   CAfile: /etc/ssl/certs/ca-certificates.crt
  CApath: /etc/ssl/certs
} [5 bytes data]
* TLSv1.3 (OUT), TLS handshake, Client hello (1):
} [512 bytes data]
* TLSv1.3 (IN), TLS handshake, Server hello (2):
{ [106 bytes data]
* TLSv1.2 (IN), TLS handshake, Certificate (11):
{ [3727 bytes data]
* TLSv1.2 (IN), TLS handshake, Server key exchange (12):
{ [300 bytes data]
* TLSv1.2 (IN), TLS handshake, Server finis

In [ ]:
AT4 = open('AT4')
jsonString = json.load(AT4)
print(jsonString.get("documents")[0])

{'address_name': '제주특별자치도 제주시 연동 281-20', 'category_group_code': 'AT4', 'category_group_name': '관광명소', 'category_name': '여행 > 관광,명소 > 동물원 > 실내동물원', 'distance': '563', 'id': '1862508988', 'phone': '064-711-1145', 'place_name': '캐니언파크 제주점', 'place_url': 'http://place.map.kakao.com/1862508988', 'road_address_name': '제주특별자치도 제주시 삼무로 51', 'x': '126.49236793947', 'y': '33.490237404658'}


In [ ]:
place_data = []


AT4 = open('AT4')
jsonString = json.load(AT4)
for i in range(len(jsonString.get("documents"))):
  place_data.append(jsonString.get("documents")[i])

MT1 = open('MT1')
jsonString = json.load(MT1)
for i in range(len(jsonString.get("documents"))):
  place_data.append(jsonString.get("documents")[i])

CT1 = open('CT1')
jsonString = json.load(CT1)
for i in range(len(jsonString.get("documents"))):
  place_data.append(jsonString.get("documents")[i])

AD5 = open('AD5')
jsonString = json.load(AD5)
for i in range(len(jsonString.get("documents"))):
  place_data.append(jsonString.get("documents")[i])

FD6 = open('FD6')
jsonString = json.load(FD6)
for i in range(len(jsonString.get("documents"))):
  place_data.append(jsonString.get("documents")[i])
  
CE7 = open('CE7')
jsonString = json.load(CE7)
for i in range(len(jsonString.get("documents"))):
  place_data.append(jsonString.get("documents")[i])



In [ ]:
print(place_data)

[{'address_name': '제주특별자치도 제주시 연동 281-20', 'category_group_code': 'AT4', 'category_group_name': '관광명소', 'category_name': '여행 > 관광,명소 > 동물원 > 실내동물원', 'distance': '563', 'id': '1862508988', 'phone': '064-711-1145', 'place_name': '캐니언파크 제주점', 'place_url': 'http://place.map.kakao.com/1862508988', 'road_address_name': '제주특별자치도 제주시 삼무로 51', 'x': '126.49236793947', 'y': '33.490237404658'}, {'address_name': '제주특별자치도 제주시 연동 273', 'category_group_code': 'AT4', 'category_group_name': '관광명소', 'category_name': '여행 > 관광,명소 > 테마거리', 'distance': '773', 'id': '21811778', 'phone': '', 'place_name': '누웨마루거리', 'place_url': 'http://place.map.kakao.com/21811778', 'road_address_name': '', 'x': '126.490657044139', 'y': '33.48601183492031'}, {'address_name': '제주특별자치도 제주시 오라이동 1758', 'category_group_code': 'AT4', 'category_group_name': '관광명소', 'category_name': '여행 > 관광,명소 > 테마파크', 'distance': '1263', 'id': '965883722', 'phone': '070-4647-0559', 'place_name': '바운스슈퍼파크 제주센터', 'place_url': 'http://place.map.kakao.

In [ ]:
with torch.no_grad(): # 모델을 학습하는 것이 아니므로 gradient 계산을 할 필요가 없음
    for i in range(len(place_data)):

        type_converter = {'AT4':0,'MT1':1,'CT1':2,'AD5':3,'FD6':4, 'CE7':5}
        group_code_raw = place_data[i].get("category_group_code")
        group_code = type_converter[group_code_raw]
        


        time_converter = {'아침':0,'오전':1,'점심':2,'오후':3,'저녁':4,'밤':5 }
        time_raw = '오후'
        time = time_converter[time_raw]

        distance = place_data[i].get("distance")



        group_code = nn.functional.one_hot(torch.tensor(group_code),6)
        time = nn.functional.one_hot(torch.tensor(time),6)
            
            
        feature = []

        for j in range(len(group_code)):
          feature.append(group_code[j])
        for j in range(len(time)):
          feature.append(time[j])
        feature.append(float(distance)*0.0001)

        
        inputs = feature
        inputs = torch.tensor([inputs], dtype = torch.float32)
        inputs = inputs.to(device)

        predicted = model(inputs)

        place_data[i]["score"] = predicted.item()
        group_code_name = place_data[i].get("category_group_name")
        #print(i, group_code_name,time_raw,distance, place_data[i]["score"])

ret = sorted(place_data, key=lambda place_data: place_data["score"], reverse=True)
for i in range(10):
  print(ret[i])

{'address_name': '제주특별자치도 제주시 연동 322-2', 'category_group_code': 'CT1', 'category_group_name': '문화시설', 'category_name': '문화,예술 > 문화시설 > 전시관', 'distance': '284', 'id': '2080157074', 'phone': '', 'place_name': '스마트그리드와 4차산업혁명 비전홀', 'place_url': 'http://place.map.kakao.com/2080157074', 'road_address_name': '제주특별자치도 제주시 문연로 36', 'x': '126.501278782819', 'y': '33.4890665346571', 'score': 9.285937309265137}
{'address_name': '제주특별자치도 제주시 오라삼동 2845', 'category_group_code': 'AT4', 'category_group_name': '관광명소', 'category_name': '여행 > 관광,명소 > 오름', 'distance': '525', 'id': '11043050', 'phone': '', 'place_name': '가삿기오름', 'place_url': 'http://place.map.kakao.com/11043050', 'road_address_name': '', 'x': '126.50387824019413', 'y': '33.489100229785734', 'score': 8.924811363220215}
{'address_name': '제주특별자치도 제주시 연동 281-20', 'category_group_code': 'AT4', 'category_group_name': '관광명소', 'category_name': '여행 > 관광,명소 > 동물원 > 실내동물원', 'distance': '563', 'id': '1862508988', 'phone': '064-711-1145', 'place_name':